#### Libraries & Settings

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime

### Data Loading

data needed : data des 3 arcs / meteo / mamoun's data pour les 3 arcs

In [ ]:
from google.colab import files
uploaded = files.upload()
#les fichiers de données externes à charger sont dans le dossier data/prediction 
#3 fichiers excel add_data 
#fichier meteo historique 
#fichier meteo prediction 
#fichier chantier 

In [ ]:
def fetch_data(arc):
    """
    Charge les données de l'arc spécifié à partir du fichier csv.
    -------
    Arg(s):
     - arc: <str> nom de l'arc ("champs", "convention" ou "sts).
    """
    try:
        df = pd.read_csv(f"data_{arc}_cleaned.csv", encoding="utf-8")
        return df
    except:
        noeud_amont = {
            "champs": "Av_Champs_Elysees-Washington",
            "convention": "Lecourbe-Convention",
            "sts": "Sts_Peres-Voltaire"
        }
        noeud_aval = {
            "champs": "Av_Champs_Elysees-Berri",
            "convention": "Convention-Blomet",
            "sts": "Sts_Peres-Universite"
        }

        df = pd.read_csv(f"data_{arc}.csv", sep=";", encoding="utf-8")
        useful_cols = ["Libelle",
                       "Date et heure de comptage",
                       "Débit horaire",
                       "Taux d'occupation",
                       "Libelle noeud amont",
                       "Libelle noeud aval"]
        df = df[useful_cols]
        df.columns = ["libelle", "date_heure", "debit", "taux", "lib_noeud_amont", "lib_noeud_aval"]

        # Filtre sur les noeuds
        df = df.loc[(df.lib_noeud_amont==noeud_amont[arc])&(df.lib_noeud_aval==noeud_aval[arc])]

        return df[["date_heure", "debit", "taux"]].sort_values("date_heure")

In [ ]:
df_champs = fetch_data("champs")
print(df_champs.shape)
df_champs.describe()

(9668, 3)


,debit,taux
count,9591.000000,9583.000000
mean,708.342300,13.088821
std,415.540502,11.356201
min,0.000000,0.000000
25%,296.000000,3.544170
50%,774.000000,10.690000
75%,1073.000000,19.205000
max,1610.000000,84.668890


In [ ]:
df_conv = fetch_data("convention")
print(df_conv.shape)
df_conv.head()

(9671, 3)


,date_heure,debit,taux
122145,2019-11-01T04:00:00+01:00,323.0,1.67722
122146,2019-11-01T05:00:00+01:00,272.0,1.41056
18265,2019-11-01T06:00:00+01:00,240.0,1.35667
18267,2019-11-01T07:00:00+01:00,216.0,1.14056
18269,2019-11-01T08:00:00+01:00,260.0,1.85722


In [ ]:
df_sts = fetch_data("sts")
print(df_sts.shape)
df_sts.head()

(9670, 3)


,date_heure,debit,taux
13746,2019-11-01T04:00:00+01:00,259.0,3.00611
12113,2019-11-01T05:00:00+01:00,197.0,2.50778
12114,2019-11-01T06:00:00+01:00,163.0,2.05111
12115,2019-11-01T07:00:00+01:00,170.0,2.08944
12116,2019-11-01T08:00:00+01:00,225.0,2.91500


In [ ]:
# Dates manquantes
dates_conv = df_conv.date_heure.to_list()
dates_champs = df_champs.date_heure.to_list()
dates_sts = df_sts.date_heure.to_list()
print(f"Dates manquantes pour Champs : {list(set.union(set(dates_conv)-set(dates_champs), set(dates_sts)-set(dates_champs)))}")
print(f"Dates manquantes pour Sts-Peres: {list(set.union(set(dates_conv)-set(dates_sts), set(dates_champs)-set(dates_sts)))}")
print(f"Dates manquantes pour Convention: {list(set.union(set(dates_champs)-set(dates_conv), set(dates_sts)-set(dates_conv)))}")

Dates manquantes pour Champs : ['2020-07-02T04:00:00+02:00', '2020-07-02T11:00:00+02:00', '2020-07-02T07:00:00+02:00', '2020-07-02T09:00:00+02:00']
Dates manquantes pour Sts-Peres: ['2020-07-02T09:00:00+02:00', '2020-07-02T07:00:00+02:00']
Dates manquantes pour Convention: ['2020-07-02T11:00:00+02:00']


In [ ]:
meteo_histo = pd.read_csv('meteo_historique.csv',sep=';')
meteo_prediction = pd.read_excel('meteo_prediction.xlsx')

### Data preprocessing

In [ ]:
#functions to use
holidays19 = [datetime(2019,11,1),datetime(2019,11,11),datetime(2019,12,25)]
holidays20 = [datetime(2020,1,1),datetime(2020,4,13),datetime(2020,5,1),datetime(2020,5,8),datetime(2020,5,21),datetime(2020,6,1),datetime(2020,7,14),datetime(2020,8,15),datetime(2020,11,1),datetime(2020,11,11),datetime(2020,12,25)]
holidays = holidays19+holidays20
holidays = [x.date() for x in holidays]

study_break = [[datetime(2019,10,19).date(),datetime(2019,11,3).date()],[datetime(2019,12,21).date(),datetime(2020,1,5).date()],[datetime(2020,2,8).date(),datetime(2020,2,23).date()],[datetime(2020,4,4).date(),datetime(2020,4,19).date()],[datetime(2020,5,19).date(),datetime(2020,5,24).date()],[datetime(2020,10,17).date(),datetime(2019,11,1).date()],[datetime(2020,7,4).date(),datetime(2020,9,1).date()]]

def clean_column_names(data):
    columns = data.columns
      data.columns = [
                        ("_".join(
      col.encode("ascii", errors="ignore")
      .decode()
      .split(" "))
      .replace(",","")
      .replace("(","")
      .replace(")","")
      .replace("%","")
  ) for col in data.columns
  ]
        return data


def holidays_binary(x):
    if x in holidays :
        return(1)
    else :
        return(0)

def study_break_binary(x,break_list):
    for dates in break_list : 
        if x>=dates[0] and x<=dates[1]:
            return(1)
    return(0)

def season(x):
    if x in [1,2,3]:
        return('winter')
    elif x in [4,5,6]:
        return('spring')
    elif x in [7,8,9]:
        return('summer')
    else:
        return('autumn')



In [ ]:
chantier = pd.read_csv("chantiers-perturbants.csv",sep=";")
def preprocess_chantier(chantier):
    chantier= chantier[["Code postal de l'arrondissement",'Date de début','Date de fin','Impact sur la circulation','Niveau de perturbation']]
    chantier= chantier.rename(columns={"Code postal de l'arrondissement":'code','Date de début':'debut','Date de fin':'fin','Impact sur la circulation':'impact','Niveau de perturbation':'niveau'})

    chantier['debut'] = pd.to_datetime(chantier['debut'], format = '%Y-%m-%d %H:%M:%S',errors='ignore')
    chantier['fin'] = pd.to_datetime(chantier['fin'], format = '%Y-%m-%d %H:%M:%S',errors='ignore')
    chantier_list = chantier.values.tolist()

    return(chantier_list)




def chantier_count(x,chantier_list,code_dep):
    count = 0
    for chantier in chantier_list:
        if chantier[0] in code_dep:
            if x>=chantier[1] and x<=chantier[2]:
                if chantier[3]=='BARRAGE_TOTAL':
                    count+=1
            else :
                count+=1
                    
    return(count)

def chantier_count_champs(x,chantier_list):
    count = 0
    for chantier in chantier_list:
        if x>=chantier[1] and x<=chantier[2]:
            if chantier[3]=='BARRAGE_TOTAL':
                count+=1
            else :
                count+=1
                    
    return(count)


chantier_list = preprocess_chantier(chantier)


In [ ]:
def process_meteo(meteo_df):
  #make weather description simpler
  description_init = ['Nuages couvrant plus de la moitié du ciel pendant toute la période considérée','Nuages couvrant plus de la moitié du ciel pendant une partie de la période considérée et couvrant la moitié du ciel, ou moins, pendant l’autre partie','Nuages ne couvrant pas plus de la moitié du ciel pendant toute la période considérée','Pluie','Bruine','Brouillard ou brouillard glacé ou brume sèche épaisse','Orage(s) avec ou sans précipitations','Neige, ou pluie et neige mêlées']
  description_new = ['nuages fort','nuages moderes','nuages faibles','pluie','bruine','brouillard','orage','neige']
  meteo_df = clean_column_names(meteo_df)
  #meteo_df['date_heure'] = pd.to_datetime(meteo_df['Date'], format = '%Y-%m-%d %H:%M:%S')
  meteo_df['date_heure'] = meteo_df.Date.apply(lambda date: pd.to_datetime(str(date).split("+")[0]))

  meteo_df[['weather']]= meteo_df.Temps_pass_1.replace(description_init,description_new)
  meteo_df[['weather']]= meteo_df.weather.fillna('normal')
  meteo_df = meteo_df[['date_heure','weather','Temprature_C','Prcipitations_dans_la_dernire_heure','Vitesse_du_vent_moyen_10_mn']]
  meteo_df = meteo_df.rename(columns={'Temprature_C':'temperature','Prcipitations_dans_la_dernire_heure':'precipitations','Vitesse_du_vent_moyen_10_mn':'wind_speed'})
  return(meteo_df)


def process(df,meteo_df,chantier_list,arc,train):
  weekday=['monday', "tuesday","wednesday","thursday","friday","saturday","sunday"]
  df.date_heure = df.date_heure.apply(lambda date: pd.to_datetime(str(date).split("+")[0]))
  
  #yasmine
  df['year'] = df.date_heure.apply(lambda date: date.year)
  df['month'] = df.date_heure.apply(lambda date: date.month)
  df["weekday"] = df.date_heure.apply(lambda date: date.weekday())
  df["weekday"] = df["weekday"].replace([0,1,2,3,4,5,6], weekday)
  df["hour"] = df.date_heure.apply(lambda date: date.hour)
  
  #yasmina
  df['date'] = df['date_heure'].apply(lambda x : x.date())
  df['season'] = df['date_heure'].apply(lambda x : season(x.month))
  df['holidays'] = df['date'].apply(lambda x : holidays_binary(x))
  df['study_break'] = df['date'].apply(lambda x : study_break_binary(x,study_break))
  
  #add weather data
  df = df.merge(meteo_df, on = 'date_heure',how='left')
  df = df.sort_values(by=['date_heure'])
  #fill nan weather data
  if train == 'train' :
    df['temperature']=df.temperature.interpolate()
    df['precipitations']=df.precipitations.interpolate()
    df['wind_speed']=df.wind_speed.interpolate()
    df['weather']=df.weather.ffill(limit=1) #Forward-fill Missing Values
    df['weather']=df.weather.bfill(limit=1) #Backfill Missing Values
    df['weather']=df.weather.ffill() #for the last value of the dataframe if it's a nan
    df['weather']=df.weather.bfill() #for the first value of the dataframe if it's a nan

  if arc == 'champs':
    df['travaux_impact'] = df.date.apply(lambda x : chantier_count_champs(x,chantier_list))
  else :
    if arc == 'sts':
      code = [75006,75004,75005,75014,75009,75007,75001,75015]
    else :
      code = [75015,75007,75006,75014,75016,75007]
    df['travaux_impact'] = df.date.apply(lambda x : chantier_count(x,chantier_list,code))
    #datetime.strptime(x,"%Y-%m-%d"
  #add mamoun's data
  new_data = pd.read_excel(f"add_data_{arc}.xlsx").drop(['Events', 'Events_location'], axis=1)
  new_data['date_heure'] = new_data.date_heure.apply(lambda date: pd.to_datetime(str(date).split("+")[0]))
  df = df.merge(new_data, on = 'date_heure',how='left')

  #dummify weekday, season and weather
  df = pd.get_dummies(df, columns = ['weekday',"season","weather"] ) #, drop_first=True)

  #fill nan taux and debit
  if train == 'train':
    df['taux'] = df['taux'].interpolate()
    df['debit'] = df['debit'].interpolate().astype('float')

  return df

In [ ]:
meteo_df = process_meteo(meteo_histo)


test avec les champs elysées , faire de même avec sts et convention

In [ ]:
processed_champs = process(df_champs,meteo_df,chantier_list,"champs",'train')
processed_champs.to_csv('processed_champs.csv')

processed_conv = process(df_conv,meteo_df,chantier_list,"conv",'train')
processed_conv.to_csv('processed_conv.csv')

processed_sts = process(df_sts,meteo_df,chantier_list,"sts",'train')
processed_sts.to_csv('processed_sts.csv')

In [ ]:
processed_champs.head()

,date_heure,debit,taux,year,month,hour,date,holidays,study_break,temperature,precipitations,wind_speed,travaux_impact,Etat_trafic_metro,Etat_trafic_RERA,Etat_trafic_RERB,Manifestations,Events_encoded,Paris_respire_encoded,Confinement,Couvre_feu,weekday_friday,weekday_monday,weekday_saturday,weekday_sunday,weekday_thursday,weekday_tuesday,weekday_wednesday,season_autumn,season_spring,season_summer,season_winter,weather_Averse(s),weather_brouillard,weather_bruine,weather_neige,weather_normal,weather_nuages faibles,weather_nuages fort,weather_nuages moderes,weather_orage,weather_pluie
0,2019-11-01 04:00:00,746.0,10.98889,2019,11,4,2019-11-01,1,1,10.700000,0.0,2.4,74,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
1,2019-11-01 05:00:00,549.0,7.40722,2019,11,5,2019-11-01,1,1,10.866667,0.8,3.0,74,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
2,2019-11-01 06:00:00,503.0,7.66667,2019,11,6,2019-11-01,1,1,11.033333,1.6,3.6,74,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
3,2019-11-01 07:00:00,500.0,4.75500,2019,11,7,2019-11-01,1,1,11.200000,2.4,4.2,74,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
4,2019-11-01 08:00:00,525.0,5.37389,2019,11,8,2019-11-01,1,1,11.566667,1.8,4.5,74,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
meteo_prediction = meteo_prediction.rename(columns={'date':'date_heure'})
ds_prediction = meteo_prediction[['date_heure']]
ds_prediction.head()

,date_heure
0,2020-12-11 00:00:00
1,2020-12-11 01:00:00
2,2020-12-11 02:00:00
3,2020-12-11 03:00:00
4,2020-12-11 04:00:00


In [ ]:
prediction_champs = process(ds_prediction,meteo_prediction,chantier_list,"champs",'pred')
prediction_sts = process(ds_prediction,meteo_prediction,chantier_list,"sts",'pred')
prediction_conv = process(ds_prediction,meteo_prediction,chantier_list,"conv",'pred')


In [ ]:
#add categories of weather not in test :
categories_weather = ['weather_Averse(s)', 'weather_brouillard', 'weather_bruine','weather_neige', 'weather_normal', 'weather_nuages faibles','weather_nuages fort', 'weather_nuages moderes', 'weather_orage','weather_pluie']
for categorie in categories_weather :
    if categorie not in list(prediction_champs.columns):
        prediction_champs[categorie] = 0
        prediction_sts[categorie] = 0
        prediction_conv[categorie] = 0

In [ ]:
prediction_champs.head()

,date_heure,year,month,hour,date,holidays,study_break,temperature,wind_speed,precipitations,travaux_impact,Etat_trafic_metro,Etat_trafic_RERA,Etat_trafic_RERB,Manifestations,Events_encoded,Paris_respire_encoded,Confinement,Couvre_feu,weekday_friday,weekday_monday,weekday_saturday,weekday_sunday,weekday_tuesday,weekday_wednesday,season_autumn,weather_brouillard,weather_normal,weather_nuages faibles,weather_nuages fort,weather_nuages moderes,weather_pluie,weather_Averse(s),weather_bruine,weather_neige,weather_orage
0,2020-12-11 00:00:00,2020,12,0,2020-12-11,0,0,9.000000,13.500000,0.000000,155,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
1,2020-12-11 01:00:00,2020,12,1,2020-12-11,0,0,8.333333,15.166667,0.000000,155,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
2,2020-12-11 02:00:00,2020,12,2,2020-12-11,0,0,7.666667,16.833333,0.000000,155,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0
3,2020-12-11 03:00:00,2020,12,3,2020-12-11,0,0,7.000000,18.500000,0.000000,155,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0
4,2020-12-11 04:00:00,2020,12,4,2020-12-11,0,0,7.333333,20.000000,0.666667,155,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0


In [ ]:
prediction_conv.drop(0, axis=0, inplace=True)
new_row = prediction_conv.iloc[-1].to_list()
new_row[0] = pd.Timestamp("2020-12-17 00:00:00")
prediction_conv = prediction_conv.append(pd.DataFrame([new_row], columns=prediction_conv.columns))

prediction_conv["debit"] = 0
prediction_conv["taux"] = 0
processed_conv = pd.concat([processed_conv, prediction_conv], axis=0)
processed_conv.drop(processed_conv.loc[processed_conv.date_heure.duplicated()].index, axis=0, inplace=True)
processed_conv.to_csv("processed_conv.csv")


In [ ]:
prediction_sts.drop(0, axis=0, inplace=True)
new_row = prediction_sts.iloc[-1].to_list()
new_row[0] = pd.Timestamp("2020-12-17 00:00:00")
prediction_sts = prediction_sts.append(pd.DataFrame([new_row], columns=prediction_sts.columns))

prediction_sts["debit"] = 0
prediction_sts["taux"] = 0
processed_sts = pd.concat([processed_sts, prediction_sts], axis=0)
processed_sts.drop(processed_sts.loc[processed_sts.date_heure.duplicated()].index, axis=0, inplace=True)
processed_sts.to_csv("processed_sts.csv")

In [ ]:
print(processed_champs.shape)
print(processed_conv.shape)
print(processed_sts.shape)

(9812, 42)
(9815, 42)
(9814, 42)
